In [1]:
!pip install tensorflow keras opencv-python matplotlib scikit-learn pandas numpy kaggle


  Obtaining dependency information for kaggle from https://files.pythonhosted.org/packages/14/83/7f29c7abe0d5dc769dad7da993382c3e4239ad63e1dd58414d129e0a4da2/kaggle-1.7.4.5-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/181.2 kB ? eta -:--:--
   ---- ---------------------------------- 20.5/181.2 kB 640.0 kB/s eta 0:00:01
   --------------------------------- ------ 153.6/181.2 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 181.2/181.2 kB 2.2 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [ ]:
!kaggle datasets download -d dansbecker/food-101


In [ ]:
!unzip food-101.zip -d ./food-101

In [ ]:
data_dir = "./food-101/food-101/images/"
categories = os.listdir(data_dir)
print("Total Categories:", len(categories))
print("Sample Categories:", categories[:10])

In [ ]:
import random
from PIL import Image

sample_category = random.choice(categories)
sample_image = random.choice(os.listdir(os.path.join(data_dir, sample_category)))
img_path = os.path.join(data_dir, sample_category, sample_image)

img = Image.open(img_path)
plt.imshow(img)
plt.title(sample_category)
plt.axis("off")
plt.show()

In [ ]:
X, y = [], []

for category in categories:
    path = os.path.join(data_dir, category)
    for img_name in os.listdir(path)[:200]:   # limit to 200 per class for faster training
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        X.append(img)
        y.append(category)

X = np.array(X) / 255.0
y = LabelBinarizer().fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

datagen.fit(X_train)

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(len(categories), activation="softmax")
])

model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test),
                    epochs=10)


In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)

plt.plot(history.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"], label="Val Acc")
plt.legend()
plt.show()


In [10]:
calorie_dict = {
    "apple_pie": 300,
    "pizza": 285,
    "ice_cream": 210,
    "fried_rice": 250,
    "donuts": 195,
    # ... add for all 101 classes
}


In [11]:
def predict_food(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)
    class_idx = np.argmax(pred)
    class_name = list(calorie_dict.keys())[class_idx]

    calories = calorie_dict.get(class_name, "Unknown")

    print(f"Predicted Food: {class_name}, Estimated Calories: {calories} kcal")


In [ ]:
test_img = "./food-101/food-101/images/pizza/1005649.jpg"
predict_food(test_img)
